In [ ]:

%load_ext autoreload
%matplotlib inline
%autoreload 2
%config Completer.use_jedi = True
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
import scalp
from scalp.output import draw

In [ ]:
datasets = scalp.data.loaddata_scib(scalp.test_config.scib_datapath,  maxdatasets=10,  maxcells = 1000 )
dataset = datasets[1]

# scanorama

In [ ]:
import scanpy as sc

def scanorama(dataset, dim = 2):
    dataset = scalp.pca.pca(dataset)
    dataset = scalp.mnn.scanorama(dataset)
    [ds.obsm.pop('umap', None)for ds in dataset]
    scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='scanorama', dim= dim)
    return scalp.transform.stack(dataset)

st = scanorama(dataset)
sc.pl.umap(st, color=['batch', 'label'])

# UMAP ONLY 

In [ ]:
import scanpy as sc
def umaponly(dataset,dim=2):
    dataset = scalp.pca.pca(dataset)
    [ds.obsm.pop('umap', None)for ds in dataset]
    scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='pca40', dim =dim)
    return scalp.transform.stack(dataset)
    
stack = umaponly(dataset)
sc.pl.umap(stack, color=['batch', 'label'])

In [ ]:
import scanpy as sc
def bbknn(dataset, dim = 2):
    dataset = scalp.pca.pca(dataset)
    [ds.obsm.pop('umap', None)for ds in dataset]
    dataset = scalp.mnn.bbknnwrap(dataset, dim = dim)
    stack = scalp.transform.stack(dataset)
    return stack

stack = bbknn(dataset)
sc.pl.umap(stack, color=['batch', 'label'])

# Combat 

In [ ]:
import scanpy as sc
def combat(dataset, dim = 2):
    [ds.obsm.pop('umap', None)for ds in dataset]
    dataset = scalp.mnn.combat(dataset)
    scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='combat', dim =dim)
    return scalp.transform.stack(dataset)
stack= combat(dataset)
sc.pl.umap(stack, color=['batch', 'label'])

# SCALP

In [ ]:
import scanpy as sc
def Scalp(dataset, dim = 2):
    parm = {'intra_neigh': 8, 'intra_neighbors_mutual': False, 'inter_neigh': 4, 'add_tree': True,
                  'copy_lsa_neighbors': False, 'inter_outlier_threshold': 0.95, 'inter_outlier_probabilistic_removal': False}
    dataset, grap = scalp.mkgraph(dataset,**parm)
    scalp.umapwrap.graph_umap(dataset,grap,label = 'umap', n_components = dim)
    return scalp.transform.stack(dataset)
    
stack = Scalp(dataset)
sc.pl.umap(stack, color=['batch', 'label'])

In [ ]:
import ubergauss.tools as ut
from lmz import *

funcs = [scanorama, umaponly, bbknn, combat, Scalp] 

fuid = Range(funcs)
dataid = Range(datasets) 
tasks = [(f,d) for f in fuid for d in dataid]

def run(fd):
    f,d = fd
    fun = funcs[f]
    dat = datasets[d]
    stack = fun(dat,10)
    return stack
    
mydata = ut.xxmap(run, tasks) 

In [ ]:
from scib.metrics import metrics_slim
def score(dataset):
    # ds2 = dataset.copy()
    # ds2.X = ds2.obsm['umap']
    return metrics_slim(dataset, dataset, 'batch', 'label', embed = 'umap',verbose = True)
    
print(score(mydata[0]))